# 1. Introduction

## Atoms of Computation

In [ ]:
#pip install -q pylatexenc

In [1]:
from qiskit import QuantumCircuit
# Create quantum circuit with 3 qubits and 3 classical bits
# (we'll explain why we need the classical bits later)
qc = QuantumCircuit(3, 3)
qc.draw()  # returns a drawing of the circuit

q_0: 
     
q_1: 
     
q_2: 
     
c: 3/

Now, measure the time and record the result for Qubits using *measure()*

In [8]:
from qiskit import QuantumCircuit
qc = QuantumCircuit(3, 3)
# measure qubits 0, 1 & 2 to classical bits 0, 1 & 2 respectively
qc.measure([0,1,2], [0,1,2])
qc.draw()

┌─┐      
q_0: ┤M├──────
     └╥┘┌─┐   
q_1: ─╫─┤M├───
      ║ └╥┘┌─┐
q_2: ─╫──╫─┤M├
      ║  ║ └╥┘
c: 3/═╩══╩══╩═
      0  1  2

Now use the simulator (which is only capable of ~30 qubits) this can be used to speed up the process

Simply use .run() after defined simulator to run simulation

In [9]:
from qiskit.providers.aer import AerSimulator
sim = AerSimulator()  # make new simulator object

job = sim.run(qc)      # qc is the circuit defined above
result = job.result()  # get the results
result.get_counts()    # {bit strings : # of times it was measured}
                       # At the start, Qubits should always be 0


{'000': 1024}

### NOT gate

In [10]:
#qc = QuantumCircuit(3, 3)
qc.x([0,1])  # Perform X-gates (NOT Gate) on qubits 0 & 1 (switching 0 <--> 1) for the first two qubits
qc.measure([0,1,2], [0,1,2])
qc.draw()    # returns a drawing of the circuit

┌─┐┌───┐        ┌─┐   
q_0: ┤M├┤ X ├────────┤M├───
     └╥┘└┬─┬┘┌───┐   └╥┘┌─┐
q_1: ─╫──┤M├─┤ X ├────╫─┤M├
      ║  └╥┘ └┬─┬┘┌─┐ ║ └╥┘
q_2: ─╫───╫───┤M├─┤M├─╫──╫─
      ║   ║   └╥┘ └╥┘ ║  ║ 
c: 3/═╩═══╩════╩═══╩══╩══╩═
      0   1    2   2  0  1

In [11]:
job = sim.run(qc)      # run the experiment
result = job.result()  # get the results
result.get_counts()    # interpret the results as a "counts" dictionary

{'011': 1024}

### Adder circuit

When calculating addition, the algorithm goes as follows:

1. Encode the input
2. Execute the algorithm
3. Extract the result

0 + 0 = 00

0 + 1 = 1 + 0 = 01

**1 + 1 = 10** --> Example below

To get this part of our solution correct, we need something that can figure out whether two bits are different or not. Traditionally, in the study of digital computation, this is called an XOR gate.

q0,q1 - for the input / executing the calculation

q2,q3 - display the output

![alt text](../assets/2.png)

It's called a CNOT, or Controlled-NOT gate in Quantum Computing. One acts as the control qubit (this is the one with the little dot). The other acts as the target qubit (with the big circle and cross - kind of like a target mark).


In [12]:
# Create quantum circuit with 2 qubits and 2 classical bits
qc = QuantumCircuit(2, 2)

qc.x(0)
qc.cx(0,1)  # CNOT controlled by qubit 0 and targeting qubit 1 
qc.measure([0,1], [0,1])
display(qc.draw())     # display a drawing of the circuit

job = sim.run(qc)      # run the experiment
result = job.result()  # get the results - A XOR B

print("Result: ", result.get_counts())

┌───┐     ┌─┐   
q_0: ┤ X ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1

Result:  {'11': 1024}


After the first half (RHS), we need to look at the LHS as well:

To calculate this part of the output, we could just get our computer to look at whether both of the inputs are 1. If they are — and only if they are — we need to do a NOT gate on qubit 3. That will flip it to the required value of 1 for this case only, giving us the output we need.

This will perform a NOT on the target qubit only when both controls are in state 1. This new gate is called the **Toffoli** gate

![alt text](../assets/3.png)

In [13]:
qc = QuantumCircuit(4,2)
qc.cx(0,2)
qc.cx(1,2)
qc.ccx(0,1,3)

In [14]:
#Final version
#M : Measure
#X : 

test_qc = QuantumCircuit(4, 2)

# First, our circuit should encode an input (here '11')
test_qc.x(0)
test_qc.x(1)

# Next, it should carry out the adder circuit we created (input q0,q1)
test_qc.cx(0,2)
test_qc.cx(1,2)
test_qc.ccx(0,1,3)

# Finally, we will measure the bottom two qubits to extract the output (output : q2,q3)
test_qc.measure(2,0)
test_qc.measure(3,1)
test_qc.draw()

┌───┐                     
q_0: ┤ X ├──■─────────■────────
     ├───┤  │         │        
q_1: ┤ X ├──┼────■────■────────
     └───┘┌─┴─┐┌─┴─┐  │  ┌─┐   
q_2: ─────┤ X ├┤ X ├──┼──┤M├───
          └───┘└───┘┌─┴─┐└╥┘┌─┐
q_3: ───────────────┤ X ├─╫─┤M├
                    └───┘ ║ └╥┘
c: 2/═════════════════════╩══╩═
                          0  1

In [15]:
job = sim.run(test_qc)  # run the experiment
result = job.result()   # get the results
result.get_counts()     # 01 + 01 = 10

{'10': 1024}

# Entanglement

Creating the CX and CZ gate

In [1]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

qc = QuantumCircuit(2)

# This calculates what the state vector of our qubits would be
# after passing through the circuit 'qc'
ket = Statevector(qc)

# The code below writes down the state vector.
# Since it's the last line in the cell, the cell will display it as output
ket.draw()

'Statevector([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],\n            dims=(2, 2))'

This will return |00⟩

In [2]:
#1. CX gate - working on the "control" and "target"
    # NO EFFECT (0 - control, 1 - target)

qc.cx(0,1)
ket = Statevector(qc)
ket.draw()

'Statevector([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],\n            dims=(2, 2))'

In [ ]:
#2. CX gate - working on the "control" and "target" Reversed
    # NO EFFECT (0 - target, 1 - control)
qc.cx(1,0)

ket = Statevector(qc)
ket.draw()

This underwhelming performance from the cx is actually to be expected. We can interpret its action as being conditional on the state of the control qubit: if the control is in state |0>, the cx does nothing. If the control is in state |1>, the cx performs an x on the target qubit.



In [3]:
# Let's create a fresh quantum circuit
qc = QuantumCircuit(2)

qc.h(1)

ket = Statevector(qc)
ket.draw()

'Statevector([0.70710678+0.j, 0.        +0.j, 0.70710678+0.j,\n             0.        +0.j],\n            dims=(2, 2))'